In [ ]:
# in case of lack of librosa library --> use !pip install librosa 

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import os

In [ ]:
# Before reading dataset it's better the visit that page: https://zenodo.org/record/1188976#.Yblv3r3P3IU


#Each of the RAVDESS files has a unique filename. 
#The filename consists of a 7-part numerical identifier (e.g., 03-01-03-01-01-01-01.wav). 
#These identifiers define the stimulus characteristics: 

#Filename identifiers 

#Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
#Vocal channel (01 = speech, 02 = song).
#Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
#Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
#Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
#Repetition (01 = 1st repetition, 02 = 2nd repetition).
#Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [ ]:
# In our study we only take "Emotion" and "Actor" and "Gender" part...

# 2-class situation -- only 2 labels male and female ... 
#This situaition answer this question--> What is the gender of speaker?

# 8-class situation -- only 8 labels like fear,happy ... 
#This situaition answer this question--> What is the fealing of speaker?

#16-class situdation -- multilabel for every feelings like fear_male,fear_female,happy_male,_happy female....... 
#This situaition answer this question--> What is the fealing and gender of speaker?

# For further study I want to take #Emotional intensity" as well...


In [ ]:
# reading dataset part

#for file in glob.iglob(/Users/alpas/Downloads/Ravdess/Actor_*/*.wav")
   # print(file)
###- -------for another way of reading is glob library------------------
########################################################################

main= r"C:\Users\alpas\Downloads\Ravdess"

dataset = os.listdir(main)  # includes file names in main dataset folder --> 24 actors
dataset.sort()

emotion = []
gender = []
path = []
for i in dataset:
    fname = os.listdir(main +"\\"+ i)
    for f in fname:
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(main + "\\"+i + '\\' +  f)

In [ ]:
############# DATA EDITING PART  #############   2 CLASS SITUATION #######################    
    # after reading it, trying to put these in to table/matrix
dataset2C_df = pd.DataFrame(gender)
        # naming the feature
dataset2C_df.columns = ['gender']

    # path is also important, when extracting features, it'll show the files
dataset2C_df = pd.concat([dataset2C_df,pd.DataFrame(path, columns = ['path'])],axis=1)

dataset2C_df.gender.value_counts()

In [ ]:
############# DATA EDITING PART  #############   8 CLASS SITUATION #######################    
    # after reading it, trying to put these in to table/matrix
dataset8C_df = pd.DataFrame(emotion)
        # there are 8 classes... Making it categorical data to make more sense
dataset8C_df = dataset8C_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
        # naming the features
dataset8C_df.columns = ['emotion']

    # path is also important, when extracting features, it'll show the files
dataset8C_df = pd.concat([dataset8C_df,pd.DataFrame(path, columns = ['path'])],axis=1)

dataset8C_df.emotion.value_counts()

In [ ]:
  ############# DATA EDITING PART  #############   16 CLASS SITUATION #######################
    ## Note: There are 8 different emotion labels and 2 gender labels. By combining these two feature I created 16 class
    ## Eg. neutral --> neutral_male and neutral_female
    
    
    
    # after reading it, trying to put these in to table/matrix
dataset16C_df = pd.DataFrame(emotion)
        # there are 8 classes... Making it categorical data to make more sense
dataset16C_df = dataset16C_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
        # there are also another feature on this data set : male or female?
dataset16C_df = pd.concat([pd.DataFrame(gender),dataset16C_df],axis=1)
        # naming the features
dataset16C_df.columns = ['gender','emotion']
        # making it new column that combine gender and emotion ------->>>>>> WITH THIS LINE THERE BECAMES 16 CLASS
dataset16C_df['labels'] =dataset16C_df.gender + '_' + dataset16C_df.emotion 
    # path is also important, when extracting features, it'll show the files
dataset16C_df = pd.concat([dataset16C_df,pd.DataFrame(path, columns = ['path'])],axis=1)
dataset16C_df = dataset16C_df.drop(['gender', 'emotion'], axis=1)
dataset16C_df.labels.value_counts()

In [ ]:
############# DATA EDITING PART  #############   16 CLASS SITUATION #######################
### CREATING CSV FILE for 16-Class data 

df = pd.concat([dataset16C_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path16Class.csv",index=False)

In [ ]:
############# DATA EDITING PART  #############   8 CLASS SITUATION #######################
### CREATING CSV FILE for 8-class data

df = pd.concat([dataset8C_df], axis = 0)
print(df.emotion.value_counts())
df.head()
df.to_csv("Data_path8Class.csv",index=False)

In [ ]:
############# DATA EDITING PART  #############   2 CLASS SITUATION #######################
### CREATING CSV FILE for 2-class data

df = pd.concat([dataset2C_df], axis = 0)
print(df.gender.value_counts())
df.head()
df.to_csv("Data_path2Class.csv",index=False)

In [ ]:
# Data Visualisation by bar plot -- for 8 class situation
height = [96, 192, 192, 192, 192,192,192,192]
bars = ('neutral', 'calm','happy','sad','angry','fear','disgust','surprise')
y_pos = np.arange(len(bars))

# Create bars
plt.bar(y_pos, height)

# Create names on the x-axis
plt.xticks(y_pos, bars)

# Show graphic
plt.show()

In [ ]:
# Some examples from emotions ---> fear 
fname = main + '\\Actor_14\\03-01-06-02-02-01-14.wav'  
data, rate = librosa.load(fname)
plt.figure(figsize=(18, 7))  
librosa.display.waveplot(data, sr=rate,color='#00E7DB')


In [ ]:
# Some examples from emotions ---> anger
fname2 =main+ "\\Actor_08\\03-01-05-02-01-01-08.wav" 
data, rate = librosa.load(fname2)
plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=rate,color='#C00808') ## with the red colour


In [ ]:
# Some examples from emotions ---> happy
fname3 =main+ "\\Actor_01\\03-01-03-01-01-01-01.wav" 
data, rate = librosa.load(fname2)
plt.figure(figsize=(18, 7))
librosa.display.waveplot(data, sr=rate,color='#F19C0E')

In [ ]:
#I've created CSV for all situation... Depends on your wish you can use it (2 , 8 or 16-class)

############################## !!!IMPORTANT!!!  ###########################
#For proper work of code,from this point after you chose dataset you should know that...
#... you can't use data combination part with others. What I mean if you choose 8-class data here...
#...in the next section you should run the relevant combination and training with the code.

# Ex: Data_path8Class ---> line with that comment "### FOR COMBINATION OF FEATURES ####  8 Classes" and -->
# ### TEST TRAIN DATA SPLIT FOR 8-CLASS dataset

In [ ]:

#full_path = pd.read_csv(r"C:\Users\alpas\OneDrive\Desktop\Lectures 20-21 Autumn\(PHYS4038 UNUK)Scientific Programming in Python  (AUT1 21-22)\SPP\Data_path2Class.csv")
full_path = pd.read_csv(r"C:\Users\alpas\OneDrive\Desktop\Lectures 20-21 Autumn\(PHYS4038 UNUK)Scientific Programming in Python  (AUT1 21-22)\SPP\Data_path8Class.csv")
#full_path = pd.read_csv(r"C:\Users\alpas\OneDrive\Desktop\Lectures 20-21 Autumn\(PHYS4038 UNUK)Scientific Programming in Python  (AUT1 21-22)\SPP\Data_path16Class.csv")


In [ ]:
# This feature extraction function is for single audio file to visualize features 
def extract_feature(path):
    
    X,_ = librosa.load(path)
    stft = librosa.amplitude_to_db(abs(librosa.stft(X)))
    mfcc = librosa.feature.mfcc(y=X, sr=rate, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=X, sr=13)
    zeroC= librosa.feature.zero_crossing_rate(y=X)
    return stft,mfcc,chroma,zeroC

In [ ]:
path = main +"\\Actor_08\\03-01-05-02-02-01-08.wav"
X, rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)  
stft,mfcc,chroma,zeroC = extract_feature(path)


In [ ]:
# Zero Crossing Rate
plt.figure(figsize=(15, 7))
librosa.display.specshow(zeroC, x_axis='time')
plt.colorbar()
plt.title('ZeroC')
plt.tight_layout()
plt.show()

In [ ]:
# MFCC
plt.figure(figsize=(20, 30))
plt.subplot(3,1,1)
librosa.display.specshow(mfcc, x_axis='time')
plt.ylabel('mfcc')
plt.title("MFCC")
plt.colorbar()

In [ ]:
#stft
plt.figure(figsize=(10, 10))
librosa.display.specshow(stft, x_axis='time')
plt.ylabel('stft')
plt.title('STFT')
plt.colorbar()

In [ ]:
#Chroma
plt.figure(figsize=(15, 4))
librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chroma')
plt.tight_layout()
plt.show()

In [ ]:
########## FEATURE EXTRACTION FUNCTIONS ################# (MFCC,STFT,CHROMA,MEL,ZCR)

In [ ]:
def mfcc_feature_exract():
    mf = pd.DataFrame(columns=['MFCC feature'])

    counter=0
    for index,path in enumerate(full_path.path):
        
        X, rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=22050,offset=0.5)
        rate = np.array(rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=rate, n_mfcc=20).T,axis=0)  # mean as the feature
        mf.loc[counter] = [mfccs]
        counter+=1
            
    print(len(mf))        
    return mf

In [ ]:

def stft_feature_exract():
    stf = pd.DataFrame(columns=['STFT feature'])

    # loop feature extraction over the entire dataset
    counter=0
    for index,path in enumerate(full_path.path):
        
        X, rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=22050,offset=0.5)
        rate = np.array(rate)
        stft = np.mean(librosa.amplitude_to_db(abs(librosa.stft(X,hop_length=64))).T,axis=0)  # mean as the feature
        stf.loc[counter]=[stft]
        counter+=1
            
    print(len(stf))        
    return stf

In [ ]:
def chroma_feature_exract():
    chr = pd.DataFrame(columns=['CHROMA feature'])

    # loop feature extraction over the entire dataset
    counter=0
    for index,path in enumerate(full_path.path):
        
        X, rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=22050,offset=0.5)
        rate = np.array(rate)
        chroma = np.mean(librosa.feature.chroma_stft(y=X, sr=rate,n_chroma=20).T,axis=0)  # mean as the feature
        chr.loc[counter]=[chroma]
        counter+=1
    
    print(len(chr))        
    return chr

In [ ]:
def mel_feature_exract():
    me = pd.DataFrame(columns=['MEL feature'])

    # loop feature extraction over the entire dataset
    counter=0
    for index,path in enumerate(full_path.path):
        
        X, rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=22050,offset=0.5)
        rate = np.array(rate)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=rate,n_mels=20).T, axis=0)  # mean as the feature
        me.loc[counter]=[mel]
        counter+=1
    
    print(len(me))        
    return me

In [ ]:
def zcr_feature_exract():
    zcr = pd.DataFrame(columns=['ZCR feature'])

    # loop feature extraction over the entire dataset
    counter=0
    for index,path in enumerate(full_path.path):
        
        X, rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=22050,offset=0.5)
        rate = np.array(rate)
        zeroC = np.mean(librosa.feature.zero_crossing_rate(y=X).T, axis=0)  # mean as the feature
        zcr.loc[counter]=[zeroC]
        counter+=1
        
    print(len(zcr))        
    return zcr

In [ ]:
########## FEATURE EXTRACTIONS ################# (MFCC,STFT,CHROMA,MEL,ZCR)

In [ ]:
Mfcc_f= mfcc_feature_exract()
Mfcc_feature = pd.concat([full_path,pd.DataFrame(Mfcc_f['MFCC feature'].values.tolist())],axis=1)


In [ ]:
Mfcc_feature

In [ ]:
zcr_f= zcr_feature_exract()
zcr_feature= pd.concat([full_path,pd.DataFrame(zcr_f['ZCR feature'].values.tolist())],axis=1)


In [ ]:
zcr_feature

In [ ]:
mel_f= mel_feature_exract()
mel_feature = pd.concat([full_path,pd.DataFrame(mel_f['MEL feature'].values.tolist())],axis=1)


In [ ]:
mel_feature

In [ ]:
CH_f= chroma_feature_exract()
CH_feature=pd.concat([full_path,pd.DataFrame(CH_f['CHROMA feature'].values.tolist())],axis=1)

In [ ]:
CH_feature

In [ ]:
stft_f= stft_feature_exract()
stft_feature=pd.concat([full_path,pd.DataFrame(stft_f['STFT feature'].values.tolist(),)],axis=1)


In [ ]:
stft_feature

In [ ]:
### FOR COMBINATION OF FEATURES ####  2 Classes
### usage of this part pd.concat([feature1,feature2,feature3........], axis=1)
A=pd.concat([Mfcc_feature,stft_feature], axis=1)
B=A.drop(columns=['path'])  # to avoid dublicate of path columns
B=B.drop(columns=['gender']) #  # to avoid dublicate of emotion columns
new_feature_set2=full_path.join(B)
new_feature_set2

In [ ]:
### FOR COMBINATION OF FEATURES ####  8 Classes
### usage of this part pd.concat([feature1,feature2,feature3........], axis=1)
A=pd.concat([Mfcc_feature,CH_feature], axis=1)
B=A.drop(columns=['path'])  # to avoid dublicate of path columns
B=B.drop(columns=['emotion']) #  # to avoid dublicate of emotion columns
new_feature_set8=full_path.join(B)
new_feature_set8

In [ ]:
### FOR COMBINATION OF FEATURES ####  16 Classes
### usage of this part pd.concat([feature1,feature2,feature3........], axis=1)
A=pd.concat([Mfcc_feature,stft_feature], axis=1)
B=A.drop(columns=['path'])  # to avoid dublicate of path columns
B=B.drop(columns=['labels']) #  # to avoid dublicate of labels columns
new_feature_set8=full_path.join(B)
new_feature_set8

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os


In [ ]:
### TEST TRAIN DATA SPLIT FOR 2-CLASS dataset
### which feature set do you want to use? for individual features use name defined above like "Mfcc_feature"
# for combination of feature set use new_feature_set2,new_feature_set8 or new_feature_set16
#  train_test_split(nameoftheset.drop(['gender','path'],axis=1)
                                        ## nameoftheset.emotion
X_train, X_test, y_train, y_test = train_test_split(stft_feature.drop(['gender','path'],axis=1)
                                                    , stft_feature.gender
                                                    , test_size=0.20
                                                    , shuffle=True
                                                    , random_state=0)
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)



X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

## for softmax normalization uncomment below

####   SOFTMAX NORMALIZATION  --->  np.exp(x)/sum(np.exp(x))
#X_train=np.exp(X_train)/sum(np.exp(X_train))
#X_train

In [ ]:
### TEST TRAIN DATA SPLIT FOR 8-CLASS dataset
### which feature set do you want to use? for individual features use name defined above like "Mfcc_feature"
# for combination of feature set use new_feature_set2,new_feature_set8 or new_feature_set16
#  train_test_split(nameoftheset.drop(['emotion','path'],axis=1)
                                        ## nameoftheset.emotion
X_train, X_test, y_train, y_test = train_test_split(new_feature_set8.drop(['emotion','path'],axis=1)
                                                    , new_feature_set8.emotion
                                                    , test_size=0.20
                                                    , shuffle=True
                                                    , random_state=0)
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)



X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

## for softmax normalization uncomment below

####   SOFTMAX NORMALIZATION  --->  np.exp(x)/sum(np.exp(x))
#X_train=np.exp(X_train)/sum(np.exp(X_train))
#X_train

In [ ]:
### TEST TRAIN DATA SPLIT FOR 16-CLASS dataset
### which feature set do you want to use? for individual features use name defined above like "Mfcc_feature"
# for combination of feature set use new_feature_set2, new_feature_set8 or new_feature_set16
#  train_test_split(nameoftheset.drop(['emotion','labels'],axis=1)
                                        ## nameoftheset.labels
X_train, X_test, y_train, y_test = train_test_split(new_feature_set16.drop(['emotion','labels'],axis=1)
                                                    , new_feature_set16.labels
                                                    , test_size=0.20
                                                    , shuffle=True
                                                    , random_state=0)

# This is standard normalization
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

## for softmax normalization uncomment below

####   SOFTMAX NORMALIZATION  --->  np.exp(x)/sum(np.exp(x))
#X_train=np.exp(X_train)/sum(np.exp(X_train))
#X_train

In [ ]:
##### MLP Classifier #####

from sklearn.neural_network import MLPClassifier
model=MLPClassifier(alpha=0.01, batch_size=128, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)



In [ ]:
#Calculate the accuracy 
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: %{:.2f}".format(accuracy*100))

In [ ]:
### SVM Classifier

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)
accuracy2=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: %{:.2f}".format(accuracy2*100))

In [ ]:
### KNN Classifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train,y_train)
print('Score:', neigh.score(X_train,y_train))